In [143]:
import torch
import torch.nn as nn
from IPython.display import display, Math, Latex

class Layer1(nn.Module):
    def __init__(self):
        super(Layer1, self).__init__()

    def forward(self, t):
        out = 2*t**2
        return out
    
class Layer2(nn.Module):
    def __init__(self):
        super(Layer2, self).__init__()
        self.param = nn.Parameter(torch.tensor(2,dtype=torch.float32),requires_grad=True) 

    def forward(self, t):
        out = (self.param * t)**3
        return out.sum()
    
class model(nn.Module):
    def __init__(self):
        super(model, self).__init__()
        self.layer1 = Layer1()
        self.layer2 = Layer2()

        
        self.layer1.register_forward_hook(self.save_activations)
        self.layer1.register_backward_hook(self.save_gradients)

    def save_activations(self, module, input, output):
        self.activations = output

    def save_gradients(self, module, grad_in, grad_out):
        self.gradients = grad_out

    def forward(self, t):
        out = self.layer1(t)
        out = self.layer2(out)
        return out

model = model()  

In [161]:
t = torch.tensor([2,4], dtype=torch.float32, requires_grad=True)
A = 2*t**2
P = 2

print(f'output: {((P*A)**3).sum()}')
print(f'grad P: {(3*A**3*P**2).sum()}')
print(f'grad A: {(3*P**3*A**2)}')

output: 266240.0
grad P: 399360.0
grad A: tensor([ 1536., 24576.], grad_fn=<MulBackward0>)


In [162]:
## Output from layer1
display(Math(r't = [2,4]'))

display(Math(r'2t^2'))
display(Math(r'Z = (PA)^3 = 266240.0'))
print('Where A = 2t^2')
display(Math(r'\frac{dZ}{dA} = 3P^3A^2 = [ 1536., 24576.]'))
display(Math(r'\frac{dZ}{dP} = 3A^3P^2 = 399360.0'))



<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Where A = 2t^2


<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [163]:
t = torch.tensor([2,4], dtype=torch.float32, requires_grad=True)
output = model(t)

model.zero_grad()
output.backward()
print(f'Output: {output.item()}')

Output: 266240.0


c:\Users\lahir\miniconda3\envs\visualize\Lib\site-packages\torch\nn\modules\module.py:1864: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)


In [164]:
activations = model.activations
gradients = model.gradients 
print(f'activations: {activations} gradients: {gradients}')
print(f'param grad: {model.layer2.param.grad}')

activations: tensor([ 8., 32.], grad_fn=<MulBackward0>) gradients: (tensor([ 1536., 24576.]),)
param grad: 399360.0
